In [14]:
import os

os.environ["TOGETHER_API_KEY"] = "8494e37999569861cb84f3e9a07df5b7f5bf3bb538fedca8cda75c571f8224b2"  # Replace with your actual API key

In [ ]:
pip install together

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.0/89.0 kB 4.0 MB/s eta 0:00:00


In [ ]:
from together import Together

client = Together()

response = client.chat.completions.create(
    model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
    messages=[{"role": "user", "content": "Buxoro tarixi haqida nimalarni bilasan?"}]
)
print(response.choices[0].message.content)

Buxoro - O'zbekistonning sharqiy qismidagi shahar, Buxoro viloyatining ma'muriy markazi. Aholisi 247 ming kishi (2010). Transport yo'llari tuguni. Shahar Zarafshon daryosining o'ng sohilida, Qizilqum cho'lining chekkasida joylashgan. Iqlimi keskin kontinental. 

Buxoro qadimda Sug'd, So'g'd, Sug'diya, Buxoro, Shahr-i Buxoro, Buxoro shahri, Buxoroi Sharif, Buxoro-i Sharif nomlari bilan atalgan. Buxoro haqidagi dastlabki ma'lumotlar miloddan avvalgi III asrda yunon tarixchisi Ptolemey asarlarida uchraydi. 

Buxoro - qadimiy madaniyat va savdo markazi. Shahar o'rnida odamlar miloddan avvalgi III asrdan yashay boshlagan. Miloddan avvalgi III-II asrlarda Buxoro Sug'd davlati tarkibida bo'lgan. Keyinchalik Kushonlar davlati tarkibiga kirgan. V-VI asrlarda Eftalitlar, VI-VII asrlarda Turk xoqonligi, VII-VIII asrlarda Arab xalifaligi, IX-X asrlarda Somoniylar davlati tarkibida bo'lgan. 

X asrda Buxoro Somoniylar poytaxt shahri bo'lib, unda ko'plab ilm-ma'rifat, madaniyat va savdo markazlari, 

In [ ]:
import pandas as pd
import joblib
from together import Together
import re
import os

# Initialize Together client
client = Together()

# Load bank information from text file
def load_bank_info(file_path='general_info.txt'):
    if not os.path.exists(file_path):
        return "Bank haqida ma'lumot fayli topilmadi."
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

bank_info = load_bank_info()

# Load the saved model and test data
model = joblib.load('linear_regression_model.pkl')
test_data = pd.read_csv('test_data2.csv')
features = ['Income', 'Rating', 'Cards', 'Age', 'Education', 'Gender', 'Student', 'Married', 'Ethnicity', 'Balance']

# Function to predict credit limit for a given ID
def predict_limit_by_id(input_id):
    if input_id not in test_data['ID'].values:
        return f"ID {input_id} topilmadi."
    input_data = test_data[test_data['ID'] == input_id][features]
    predicted_limit = model.predict(input_data)[0]
    return predicted_limit

# Function to detect query type
def is_credit_query(message):
    credit_keywords = ['kredit', 'qarz', 'limit', 'pul olish', 'kredit olish', 'kredit limiti']
    return any(keyword.lower() in message.lower() for keyword in credit_keywords)

# Function to generate response using Together API
def generate_response(prompt, chat_history):
    messages = chat_history + [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
        messages=messages
    )
    return response.choices[0].message.content

# Chatbot class to manage state and interactions
class BankChatbot:
    def __init__(self):
        self.chat_history = []
        self.waiting_for_id = False

    def process_message(self, user_input):
        # Append user message to chat history
        self.chat_history.append({"role": "user", "content": user_input})

        # Check if waiting for ID input
        if self.waiting_for_id:
            try:
                input_id = int(user_input.strip())
                prediction = predict_limit_by_id(input_id)
                if isinstance(prediction, str):
                    response = prediction
                else:
                    response = f"ID {input_id} uchun taxminiy kredit limiti: {prediction:.2f} USD"
                self.waiting_for_id = False
            except ValueError:
                response = "Iltimos, to'g'ri ID raqamini kiriting (faqat raqamlar)."
        else:
            # Determine query type
            if is_credit_query(user_input):
                self.waiting_for_id = True
                response = "Kredit limiti haqida ma'lumot olish uchun ID raqamingizni kiriting."
            else:
                # Use bank info for general queries
                prompt = f"Quyidagi ma'lumot asosida savolga javob bering:\n{bank_info}\n\nSavol: {user_input}"
                response = generate_response(prompt, self.chat_history)

        # Append assistant response to chat history
        self.chat_history.append({"role": "assistant", "content": response})

        # Keep chat history manageable (e.g., last 10 messages)
        if len(self.chat_history) > 10:
            self.chat_history = self.chat_history[-10:]

        return response

# Main function to run the chatbot
def main():
    chatbot = BankChatbot()
    print("Ipak Yo'li Bank Chatbotiga xush kelibsiz! Savollaringizni o'zbek tilida yozing.")
    print("Chiqish uchun 'exit' deb yozing.")

    while True:
        user_input = input("Siz: ")
        if user_input.lower() == 'exit':
            print("Xayr, yana ko'rishamiz!")
            break

        response = chatbot.process_message(user_input)
        print("Bot:", response)

if __name__ == "__main__":
    main()

Ipak Yo'li Bank Chatbotiga xush kelibsiz! Savollaringizni o'zbek tilida yozing.
Chiqish uchun 'exit' deb yozing.
Siz: salom
Bot: Salom! Sizga "Ipak Yo'li" banki haqida ma'lumot kerakmi?
Siz: ha menga ma'lumot bergin
Bot: "Ipak Yo'li" aksiyadorlik innovatsiya tijorat banki haqida quyidagi ma'lumotlarni bilib olishingiz mumkin:

1. **Tashkil etilgan yili**: 1990-yil
2. **Litsenziya**: Bank litsenziyasi asosida ishlaydi
3. **Bosh ofis**: Toshkent shahrida joylashgan
4. **Aktsiyadorlar**: Osiyo taraqqiyot banki (2013-yildan), Germaniyaning taraqqiyot moliya instituti DEG-Deutsche Investitions-und Entwicklungsgesellschaft mbH va Triodos Investment Management (2020-yildan)
5. **Filiallar**: Jami 17 ta filial, shu jumladan 9 ta viloyat filiali va 8 ta Toshkent shahar filiali
6. **Mini-banklar**: 46 ta
7. **Maxsus kassalar**: 22 ta
8. **Terminallar**: 10 mingdan ortiq
9. **Bankomatlar**: 600 dan ortiq (2019-yil davomida o'rnatilgan)
10. **Raqamli kanallar**: Ipak Yo'li Mobile, Ipak Yo'li Mobil

KeyboardInterrupt: Interrupted by user

In [ ]:
import pandas as pd
import joblib
from together import Together
import re
import os

# Initialize Together client
client = Together()

# Load bank information from text file
def load_bank_info(file_path='general_info.txt'):
    if not os.path.exists(file_path):
        return "Bank haqida ma'lumot fayli topilmadi."
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

bank_info = load_bank_info()

# Load the saved model and test data
model = joblib.load('linear_regression_model.pkl')
test_data = pd.read_csv('test_data2.csv')
features = ['Income', 'Rating', 'Cards', 'Age', 'Education', 'Gender', 'Student', 'Married', 'Ethnicity', 'Balance']

# Function to predict credit limit for a given ID
def predict_limit_by_id(input_id):
    if input_id not in test_data['ID'].values:
        return f"ID {input_id} topilmadi."
    input_data = test_data[test_data['ID'] == input_id][features]
    predicted_limit = model.predict(input_data)[0]
    return predicted_limit

# Function to detect query type
def is_credit_query(message):
    credit_keywords = ['kredit', 'qarz', 'limit', 'pul olish', 'kredit olish', 'kredit limiti']
    return any(keyword.lower() in message.lower() for keyword in credit_keywords)

# Function to generate response using Together API
def generate_response(prompt, chat_history):
    messages = chat_history + [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
        messages=messages
    )
    return response.choices[0].message.content

# Function to parse Uzbek number words to integer
def uzbek_text_to_number(text):
    """Convert Uzbek number words to integer (e.g., 'ikki yuzi oltmish yetti' to 267)."""
    number_map = {
        'nol': 0, 'bir': 1, 'ikki': 2, 'uch': 3, 'to\'rt': 4, 'besh': 5,
        'olti': 6, 'yetti': 7, 'sakkiz': 8, 'to\'qqiz': 9,
        'o\'n': 10, 'yigirma': 20, 'o\'ttiz': 30, 'qirq': 40, 'ellik': 50,
        'oltmish': 60, 'yetmish': 70, 'sakson': 80, 'to\'qson': 90,
        'yuz': 100, 'ming': 1000, 'million': 1000000
    }

    text = text.lower().replace('va', '').strip()
    words = text.split()

    if not words:
        return None

    total = 0
    current = 0
    for word in words:
        if word not in number_map:
            return None
        num = number_map[word]
        if num in [100, 1000, 1000000]:
            if current == 0:
                current = 1  # Assume 1 if no number before multiplier
            total += current * num
            current = 0
        else:
            current += num
    total += current
    return total if total > 0 else None

# Chatbot class to manage state and interactions
class BankChatbot:
    def __init__(self):
        self.chat_history = []
        self.waiting_for_id = False

    def process_message(self, user_input):
        # Append user message to chat history
        self.chat_history.append({"role": "user", "content": user_input})

        # Check if waiting for ID input
        if self.waiting_for_id:
            # Try to parse as a number directly
            try:
                input_id = int(user_input.strip())
                prediction = predict_limit_by_id(input_id)
                if isinstance(prediction, str):
                    response = prediction
                else:
                    response = f"ID {input_id} uchun taxminiy kredit limiti: {prediction:.2f} USD"
                self.waiting_for_id = False
            except ValueError:
                # Try to parse as Uzbek number words
                parsed_id = uzbek_text_to_number(user_input)
                if parsed_id is not None:
                    prediction = predict_limit_by_id(parsed_id)
                    if isinstance(prediction, str):
                        response = prediction
                    else:
                        response = f"ID {parsed_id} uchun taxminiy kredit limiti: {prediction:.2f} USD"
                    self.waiting_for_id = False
                else:
                    response = "Iltimos, to'g'ri ID raqamini kiriting (raqamlar yoki so'zlar bilan, masalan, '267' yoki 'ikki yuzi oltmish yetti')."
        else:
            # Determine query type
            if is_credit_query(user_input):
                self.waiting_for_id = True
                response = "Kredit limiti haqida ma'lumot olish uchun ID raqamingizni kiriting (raqamlar yoki so'zlar bilan, masalan, '267' yoki 'ikki yuzi oltmish yetti')."
            else:
                # Use bank info for general queries
                prompt = f"Quyidagi ma'lumot asosida savolga javob bering:\n{bank_info}\n\nSavol: {user_input}"
                response = generate_response(prompt, self.chat_history)

        # Append assistant response to chat history
        self.chat_history.append({"role": "assistant", "content": response})

        # Keep chat history manageable (e.g., last 10 messages)
        if len(self.chat_history) > 10:
            self.chat_history = self.chat_history[-10:]

        return response

# Main function to run the chatbot
def main():
    chatbot = BankChatbot()
    print("Ipak Yo'li Bank Chatbotiga xush kelibsiz! Savollaringizni o'zbek tilida yozing.")
    print("Chiqish uchun 'exit' deb yozing.")

    while True:
        user_input = input("Siz: ")
        if user_input.lower() == 'exit':
            print("Xayr, yana ko'rishamiz!")
            break

        response = chatbot.process_message(user_input)
        print("Bot:", response)

if __name__ == "__main__":
    main()

Ipak Yo'li Bank Chatbotiga xush kelibsiz! Savollaringizni o'zbek tilida yozing.
Chiqish uchun 'exit' deb yozing.
Siz: salom
Bot: Salom! Siz "Ipak Yo'li" banki haqida ma'lumot izlayapsizmi?
Siz: ha menga bank haqida ma'lumot bersangiz
Bot: "Ipak Yo'li" aksiyadorlik innovatsiya tijorat banki 1990 yildan buyon faoliyat yuritib kelmoqda. Bankning bosh ofisi Toshkent shahrida joylashgan. Bank litsenziyasi asosida ishlaydi.

Bankning tarixi:
- 1990 yilda tashkil topgan
- 2013 yildan beri Osiyo taraqqiyot banki aktsiyadori
- 2020 yilda Germaniyaning DEG-Deutsche Investitions-und Entwicklungsgesellschaft mbH va Triodos Investment Management investitsiya kiritdi

Bankning faoliyati:
- 2021 yilning birinchi yarmida O'zbekistondagi tijorat banklari faoliyatini tahlil qilish natijalariga ko'ra, 13-o'rinni egallab turibdi
- Bozor ulushi 2,03 foizni tashkil etdi

Bankning filiallari:
- Jami 17 ta filialga ega
- 9 ta filial viloyatlarda, 8 ta filial Toshkent shahrida
- 46 ta mini-bank, 22 ta maxsus k

KeyboardInterrupt: Interrupted by user

In [ ]:
import pandas as pd
import joblib
from together import Together
import re
import os

# Initialize Together client
client = Together()

# Load bank information from text file
def load_bank_info(file_path='general_info.txt'):
    if not os.path.exists(file_path):
        return "Bank haqida ma'lumot fayli topilmadi."
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

bank_info = load_bank_info()

# Load the saved model and test data
model = joblib.load('linear_regression_model.pkl')
test_data = pd.read_csv('test_data2.csv')
features = ['Income', 'Rating', 'Cards', 'Age', 'Education', 'Gender', 'Student', 'Married', 'Ethnicity', 'Balance']

# Function to predict credit limit for a given ID
def predict_limit_by_id(input_id):
    if input_id not in test_data['ID'].values:
        return f"ID {input_id} topilmadi."
    input_data = test_data[test_data['ID'] == input_id][features]
    predicted_limit = model.predict(input_data)[0]
    return predicted_limit

# Function to detect query type
def is_credit_query(message):
    credit_keywords = ['kredit', 'qarz', 'limit', 'pul olish', 'kredit olish', 'kredit limiti']
    return any(keyword.lower() in message.lower() for keyword in credit_keywords)

# Function to generate response using Together API
def generate_response(prompt, chat_history):
    messages = chat_history + [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
        messages=messages
    )
    return response.choices[0].message.content

# Function to parse Uzbek number words or digits from a sentence
def uzbek_text_to_number(text):
    """Extract and convert Uzbek number words or digits from a sentence to an integer."""
    number_map = {
        'nol': 0, 'bir': 1, 'ikki': 2, 'uch': 3, 'to\'rt': 4, 'besh': 5,
        'olti': 6, 'yetti': 7, 'sakkiz': 8, 'to\'qqiz': 9,
        'o\'n': 10, 'yigirma': 20, 'o\'ttiz': 30, 'qirq': 40, 'ellik': 50,
        'oltmish': 60, 'yetmish': 70, 'sakson': 80, 'to\'qson': 90,
        'yuz': 100, 'ming': 1000, 'million': 1000000
    }

    # Normalize text
    text = text.lower().replace('va', '').strip()

    # Try to find a direct numeric value first (e.g., "127")
    numeric_match = re.search(r'\b\d+\b', text)
    if numeric_match:
        try:
            return int(numeric_match.group())
        except ValueError:
            pass

    # Extract potential number words
    words = text.split()
    number_words = []
    for word in words:
        if word in number_map:
            number_words.append(word)
        elif re.match(r'[\d]', word):
            # Skip if word contains digits but isn't purely numeric (handled above)
            continue
        else:
            # Stop collecting if we hit a non-number word
            if number_words:
                break

    if not number_words:
        return None

    # Parse number words
    total = 0
    current = 0
    for word in number_words:
        num = number_map[word]
        if num in [100, 1000, 1000000]:
            if current == 0:
                current = 1  # Assume 1 if no number before multiplier
            total += current * num
            current = 0
        else:
            current += num
    total += current
    return total if total > 0 else None

# Chatbot class to manage state and interactions
class BankChatbot:
    def __init__(self):
        self.chat_history = []
        self.waiting_for_id = False

    def process_message(self, user_input):
        # Append user message to chat history
        self.chat_history.append({"role": "user", "content": user_input})

        # Check if waiting for ID input
        if self.waiting_for_id:
            # Try to extract a number from the input
            parsed_id = uzbek_text_to_number(user_input)
            if parsed_id is not None:
                prediction = predict_limit_by_id(parsed_id)
                if isinstance(prediction, str):
                    response = prediction
                else:
                    response = f"ID {parsed_id} uchun taxminiy kredit limiti: {prediction:.2f} USD"
                self.waiting_for_id = False
            else:
                response = "Iltimos, to'g'ri ID raqamini kiriting (raqamlar yoki so'zlar bilan, masalan, '127' yoki 'bir yuz yigirma yetti')."
        else:
            # Determine query type
            if is_credit_query(user_input):
                self.waiting_for_id = True
                response = "Kredit limiti haqida ma'lumot olish uchun ID raqamingizni kiriting (raqamlar yoki so'zlar bilan, masalan, '127' yoki 'bir yuz yigirma yetti')."
            else:
                # Use bank info for general queries
                prompt = f"Quyidagi ma'lumot asosida savolga javob bering:\n{bank_info}\n\nSavol: {user_input}"
                response = generate_response(prompt, self.chat_history)

        # Append assistant response to chat history
        self.chat_history.append({"role": "assistant", "content": response})

        # Keep chat history manageable (e.g., last 10 messages)
        if len(self.chat_history) > 10:
            self.chat_history = self.chat_history[-10:]

        return response

# Main function to run the chatbot
def main():
    chatbot = BankChatbot()
    print("Ipak Yo'li Bank Chatbotiga xush kelibsiz! Savollaringizni o'zbek tilida yozing.")
    print("Chiqish uchun 'exit' deb yozing.")

    while True:
        user_input = input("Siz: ")
        if user_input.lower() == 'exit':
            print("Xayr, yana ko'rishamiz!")
            break

        response = chatbot.process_message(user_input)
        print("Bot:", response)

if __name__ == "__main__":
    main()

Ipak Yo'li Bank Chatbotiga xush kelibsiz! Savollaringizni o'zbek tilida yozing.
Chiqish uchun 'exit' deb yozing.
Siz: salom
Bot: Salom! Siz "Ipak Yo'li" banki haqida ma'lumot izlayapsizmi?
Siz: ha menga bank haqida malumot bergin
Bot: "Ipak Yo'li" aksiyadorlik innovatsiya tijorat banki 1990 yildan buyon faoliyat yuritib kelmoqda. Bankning bosh ofisi Toshkent shahrida joylashgan. Bank litsenziyasi asosida ishlaydi.

Bankning tarixi:
- 1990 yilda tashkil topgan
- 2013 yildan beri Osiyo taraqqiyot banki aktsiyadori
- 2020 yilda Germaniyaning DEG-Deutsche Investitions-und Entwicklungsgesellschaft mbH va Triodos Investment Management investitsiya kiritdi

Bankning faoliyati:
- 2021 yilning birinchi yarmida O'zbekistondagi tijorat banklari faoliyati ko'lami bo'yicha 13-o'rinni egallab turibdi
- Bozor ulushi 2,03 foizni tashkil etdi

Bankning filiallari:
- Jami 17 ta filialga ega
- 9 ta filial viloyatlarda, 8 ta filial Toshkent shahrida
- 46 ta mini-bank, 22 ta maxsus kassalar, 10 mingdan ort

KeyboardInterrupt: Interrupted by user

In [ ]:
import pandas as pd
import joblib
from together import Together
import re
import os

# Initialize Together client
client = Together()

# Load bank information from text file
def load_bank_info(file_path='general_info.txt'):
    if not os.path.exists(file_path):
        return "Bank haqida ma'lumot fayli topilmadi."
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

bank_info = load_bank_info()

# Load the saved model and test data
model = joblib.load('linear_regression_model.pkl')
test_data = pd.read_csv('test_data2.csv')
features = ['Income', 'Rating', 'Cards', 'Age', 'Education', 'Gender', 'Student', 'Married', 'Ethnicity', 'Balance']

# Function to predict credit limit for a given ID
def predict_limit_by_id(input_id):
    if input_id not in test_data['ID'].values:
        return f"ID {input_id} topilmadi."
    input_data = test_data[test_data['ID'] == input_id][features]
    predicted_limit = model.predict(input_data)[0]
    return predicted_limit

# Function to detect query type
def is_credit_query(message):
    credit_keywords = ['kredit', 'qarz', 'limit', 'pul olish', 'kredit olish', 'kredit limiti']
    return any(keyword.lower() in message.lower() for keyword in credit_keywords)

# Function to generate response using Together API
def generate_response(prompt, chat_history):
    messages = chat_history + [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
        messages=messages
    )
    return response.choices[0].message.content

# Function to parse Uzbek number words or digits from a sentence
def uzbek_text_to_number(text):
    """Extract and convert Uzbek number words or digits from a sentence to an integer."""
    number_map = {
        'nol': 0, 'bir': 1, 'ikki': 2, 'uch': 3, 'to\'rt': 4, 'besh': 5,
        'olti': 6, 'yetti': 7, 'sakkiz': 8, 'to\'qqiz': 9,
        'o\'n': 10, 'yigirma': 20, 'o\'ttiz': 30, 'qirq': 40, 'ellik': 50,
        'oltmish': 60, 'yetmish': 70, 'sakson': 80, 'to\'qson': 90,
        'yuz': 100, 'yuzi': 100, 'ming': 1000, 'million': 1000000
    }

    # Normalize text
    text = text.lower().replace('va', '').strip()

    # Try to find a direct numeric value first (e.g., "127")
    numeric_match = re.search(r'\b\d+\b', text)
    if numeric_match:
        try:
            return int(numeric_match.group())
        except ValueError:
            pass

    # Extract potential number words
    words = text.split()
    number_words = []
    for word in words:
        if word in number_map:
            number_words.append(word)
        elif re.match(r'[\d]', word):
            # Skip if word contains digits but isn't purely numeric (handled above)
            continue
        else:
            # Stop collecting if we hit a non-number word
            if number_words:
                break

    if not number_words:
        return None

    # Parse number words
    total = 0
    current = 0
    for word in number_words:
        num = number_map[word]
        if num in [100, 1000, 1000000]:
            if current == 0:
                current = 1  # Assume 1 if no number before multiplier
            total += current * num
            current = 0
        else:
            current += num
    total += current
    return total if total > 0 else None

# Chatbot class to manage state and interactions
class BankChatbot:
    def __init__(self):
        self.chat_history = []
        self.waiting_for_id = False

    def process_message(self, user_input):
        # Append user message to chat history
        self.chat_history.append({"role": "user", "content": user_input})

        # Check if waiting for ID input
        if self.waiting_for_id:
            # Try to extract a number from the input
            parsed_id = uzbek_text_to_number(user_input)
            if parsed_id is not None:
                prediction = predict_limit_by_id(parsed_id)
                if isinstance(prediction, str):
                    response = prediction
                else:
                    response = f"ID {parsed_id} uchun taxminiy kredit limiti: {prediction:.2f} USD. Sizga bir yil muddatga ushbu miqdorda kredit berishimiz mumkin."
                self.waiting_for_id = False
            else:
                response = "Iltimos, to'g'ri ID raqamini kiriting (raqamlar yoki so'zlar bilan, masalan, '127' yoki 'bir yuz yigirma yetti')."
        else:
            # Determine query type
            if is_credit_query(user_input):
                self.waiting_for_id = True
                response = "Kredit limiti haqida ma'lumot olish uchun ID raqamingizni kiriting (raqamlar yoki so'zlar bilan, masalan, '127' yoki 'bir yuz yigirma yetti')."
            else:
                # Use bank info for general queries
                prompt = f"Quyidagi ma'lumot asosida savolga javob bering:\n{bank_info}\n\nSavol: {user_input}"
                response = generate_response(prompt, self.chat_history)

        # Append assistant response to chat history
        self.chat_history.append({"role": "assistant", "content": response})

        # Keep chat history manageable (e.g., last 10 messages)
        if len(self.chat_history) > 10:
            self.chat_history = self.chat_history[-10:]

        return response

# Main function to run the chatbot
def main():
    chatbot = BankChatbot()
    print("Ipak Yo'li Bank Chatbotiga xush kelibsiz! Savollaringizni o'zbek tilida yozing.")
    print("Chiqish uchun 'exit' deb yozing.")

    while True:
        user_input = input("Siz: ")
        if user_input.lower() == 'exit':
            print("Xayr, yana ko'rishamiz!")
            break

        response = chatbot.process_message(user_input)
        print("Bot:", response)

if __name__ == "__main__":
    main()

Ipak Yo'li Bank Chatbotiga xush kelibsiz! Savollaringizni o'zbek tilida yozing.
Chiqish uchun 'exit' deb yozing.
Siz: salom
Bot: Salom! Siz "Ipak Yo'li" banki haqida ma'lumot izlayapsizmi?
Siz: men kredit olmoqchi eidm
Bot: Kredit limiti haqida ma'lumot olish uchun ID raqamingizni kiriting (raqamlar yoki so'zlar bilan, masalan, '127' yoki 'bir yuz yigirma yetti').
Siz: mening id raqamim bir yuzi yigirma yetti
Bot: ID 127 uchun taxminiy kredit limiti: 5751.81 USD. Sizga bir yil muddatga ushbu miqdorda kredit berishimiz mumkin.
Siz: ajoyib
Bot: "Ipak Yo'li" banki haqida ma'lumot berilgan. Bank 1990 yildan buyon faoliyat yuritib kelmoqda va O'zbekistondagi tijorat banklari orasida 13-o'rinni egallab turibdi. Bankning 17 ta filiali, 46 ta mini-bank va 22 ta maxsus kassalari, 10 mingdan ortiq terminallari mavjud. Bankning raqamli kanallari qatoriga Ipak Yo'li Mobile, Ipak Yo'li Mobile biznes uchun mobil ilovalar va Internet-banking xizmati kiradi.
Siz: bankning telefon raqami nechchi?
Bot: 

KeyboardInterrupt: Interrupted by user